# SQLITE test

In [1]:
import sqlite3
import threading
import concurrent.futures

In [2]:
# user settings
yearrange = [1960, 1990]
STATUS = ('unknown', 'in progress', 'done', 'failed')

In [3]:
# setup database to keep track of downloads
con = sqlite3.connect("downloads.db")
cur = con.cursor()
cur.execute("CREATE TABLE IF NOT EXISTS downloads(year, status)")
for y in range(yearrange[0], yearrange[1]):
    cur.execute(f"""INSERT INTO downloads VALUES
        ({y}, 'unknown')""")
con.commit()
# check DB content
res = cur.execute("SELECT * FROM downloads")
res.fetchall()

[(1960, 'unknown'),
 (1961, 'unknown'),
 (1962, 'unknown'),
 (1963, 'unknown'),
 (1964, 'unknown'),
 (1965, 'unknown'),
 (1966, 'unknown'),
 (1967, 'unknown'),
 (1968, 'unknown'),
 (1969, 'unknown'),
 (1970, 'unknown'),
 (1971, 'unknown'),
 (1972, 'unknown'),
 (1973, 'unknown'),
 (1974, 'unknown'),
 (1975, 'unknown'),
 (1976, 'unknown'),
 (1977, 'unknown'),
 (1978, 'unknown'),
 (1979, 'unknown'),
 (1980, 'unknown'),
 (1981, 'unknown'),
 (1982, 'unknown'),
 (1983, 'unknown'),
 (1984, 'unknown'),
 (1985, 'unknown'),
 (1986, 'unknown'),
 (1987, 'unknown'),
 (1988, 'unknown'),
 (1989, 'unknown'),
 (1960, 'unknown'),
 (1961, 'unknown'),
 (1962, 'unknown'),
 (1963, 'unknown'),
 (1964, 'unknown'),
 (1965, 'unknown'),
 (1966, 'unknown'),
 (1967, 'unknown'),
 (1968, 'unknown'),
 (1969, 'unknown'),
 (1970, 'unknown'),
 (1971, 'unknown'),
 (1972, 'unknown'),
 (1973, 'unknown'),
 (1974, 'unknown'),
 (1975, 'unknown'),
 (1976, 'unknown'),
 (1977, 'unknown'),
 (1978, 'unknown'),
 (1979, 'unknown'),


In [28]:
# data download from noaa
def download(year):
    url = f"https://www.ncei.noaa.gov/pub/data/ghcn/daily/by_year/{year}.csv.gz"
    print(url)
    # response = requests.get(url)
    """
    if response.ok:
        filename = url.rsplit('/', 1)[1]
        print(f"data downloaded. Will be saved as {filename}")
        with open(filename, "wb") as f:
            f.write(response.content)
    else:
        print(f"An error occured while trying to retrieve the data of year {year} from the internet.")
    """
    return 5

def manage_download(year):
    status = download(year)
    print(cur,status,year)    
    cur.execute(f"UPDATE downloads SET status={status} WHERE year={year} ")
    con.commit()
    print("updated")


In [29]:
# main loop
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    executor.map(manage_download, range(yearrange[0], yearrange[1]))


https://www.ncei.noaa.gov/pub/data/ghcn/daily/by_year/1960.csv.gz
<sqlite3.Cursor object at 0x7f97b80dc940> 5 1960
https://www.ncei.noaa.gov/pub/data/ghcn/daily/by_year/1961.csv.gz
<sqlite3.Cursor object at 0x7f97b80dc940> 5 1961
https://www.ncei.noaa.gov/pub/data/ghcn/daily/by_year/1962.csv.gz
<sqlite3.Cursor object at 0x7f97b80dc940> 5 1962
https://www.ncei.noaa.gov/pub/data/ghcn/daily/by_year/1963.csv.gz
<sqlite3.Cursor object at 0x7f97b80dc940> 5 1963
https://www.ncei.noaa.gov/pub/data/ghcn/daily/by_year/1964.csv.gz
<sqlite3.Cursor object at 0x7f97b80dc940> 5 1964
https://www.ncei.noaa.gov/pub/data/ghcn/daily/by_year/1965.csv.gz
<sqlite3.Cursor object at 0x7f97b80dc940> 5 1965
https://www.ncei.noaa.gov/pub/data/ghcn/daily/by_year/1966.csv.gz
<sqlite3.Cursor object at 0x7f97b80dc940> 5 1966
https://www.ncei.noaa.gov/pub/data/ghcn/daily/by_year/1967.csv.gz
<sqlite3.Cursor object at 0x7f97b80dc940> 5 1967
https://www.ncei.noaa.gov/pub/data/ghcn/daily/by_year/1968.csv.gz
<sqlite3.Curso

In [34]:
# check DB content
res = cur.execute("SELECT * FROM downloads")
res.fetchall()

[(1960, 'unknown'),
 (1961, 'unknown'),
 (1962, 'unknown'),
 (1963, 'unknown'),
 (1964, 'unknown'),
 (1965, 'unknown'),
 (1966, 'unknown'),
 (1967, 'unknown'),
 (1968, 'unknown'),
 (1969, 'unknown'),
 (1970, 'unknown'),
 (1971, 'unknown'),
 (1972, 'unknown'),
 (1973, 'unknown'),
 (1974, 'unknown'),
 (1975, 'unknown'),
 (1976, 'unknown'),
 (1977, 'unknown'),
 (1978, 'unknown'),
 (1979, 'unknown'),
 (1980, 5),
 (1981, 'unknown'),
 (1982, 'unknown'),
 (1983, 'unknown'),
 (1984, 'unknown'),
 (1985, 'unknown'),
 (1986, 'unknown'),
 (1987, 'unknown'),
 (1988, 'unknown'),
 (1989, 'unknown'),
 (1960, 'unknown'),
 (1961, 'unknown'),
 (1962, 'unknown'),
 (1963, 'unknown'),
 (1964, 'unknown'),
 (1965, 'unknown'),
 (1966, 'unknown'),
 (1967, 'unknown'),
 (1968, 'unknown'),
 (1969, 'unknown'),
 (1970, 'unknown'),
 (1971, 'unknown'),
 (1972, 'unknown'),
 (1973, 'unknown'),
 (1974, 'unknown'),
 (1975, 'unknown'),
 (1976, 'unknown'),
 (1977, 'unknown'),
 (1978, 'unknown'),
 (1979, 'unknown'),
 (1980, 

In [33]:
manage_download(1980)

https://www.ncei.noaa.gov/pub/data/ghcn/daily/by_year/1980.csv.gz
<sqlite3.Cursor object at 0x7f97b80dc940> 5 1980
updated
